In [22]:
import nasdaqdatalink

import os
from dotenv import load_dotenv

load_dotenv()
nasdaqdatalink.read_key( os.getenv('nasdaq_key'))


In [25]:
# documentation
## https://docs.data.nasdaq.com/docs/python-time-series


data = nasdaqdatalink.get("EIA/PET_RWTC_D")


            Value
Date             
1986-01-02  25.56
1986-01-03  26.00
1986-01-06  26.53
1986-01-07  25.85
1986-01-08  25.87
...           ...
2022-02-02  88.16
2022-02-03  90.17
2022-02-04  92.27
2022-02-07  91.25
2022-02-08  89.32

[9124 rows x 1 columns]
